# Collect Masks and their Instant Feautures

In [1]:
import preprocessingNA as pp
import data2D3D
import os
import glob
import numpy as np
import logging
import nibabel as nib
import gc
from skimage import transform
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import tensorflow as tf
from torch.utils.data import SubsetRandomSampler
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import random
import pickle
import sys
%matplotlib inline
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import torch
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torch.utils import data as D
from torch import nn
from torch.nn import functional as F
from torchvision import models
import torch.optim as optim
from time import time
from data2D3D import data2D3D
from D_UNet import U_Net


logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')
nx=224;
img_T, mask_T, imageSS, num=pp.data_prepare(norm=False)
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Datacollect folder contains all subjects' ED and ES scans

In [2]:
path = 'datacollect'    
tumor_dataset = data2D3D(path, imageSS, img_T, mask_T, num, nx, aug=True)
dataloader = torch.utils.data.DataLoader(tumor_dataset,shuffle=False )


In [5]:
mskC=glob.glob(os.path.join('datacollect', 'patient???_frame??_gt.nii.gz'))
mskC.sort()
thickness=[]
for i in range(200):
    mask_name=mskC[i]
    mask = nib.load(mask_name).get_data()
    thickness.append(mask.shape[-1])

## Generate masks and save into ./datacollect/Saved folders

In [9]:
device = torch.device('cpu')
image_save_path = './datacollect/Saved/'
os.makedirs(image_save_path, exist_ok = True)
unet_model = U_Net().to(device)
state_dict = torch.load('./saved_models/WCE/Basic_Unet_epoch_066.pth',map_location=torch.device('cpu'))
unet_model.load_state_dict(state_dict)
unet_model.eval()

indexImg=0 
indexSlice=0
new_image=np.zeros([nx,nx,10])
for batch, data in enumerate(dataloader):
    
    # Data prepared to be given as input to model.
    IMG2D = data['image2D'].type(torch.float)
    IMG3D = data['image3D'].type(torch.float)
    MSK = data['mask2D'].type(torch.float)

    unet_model.eval()
    image2D = torch.clone(IMG2D).numpy()
    image3D = torch.clone(IMG3D).numpy()    
    mask = torch.clone(MSK).numpy()    
    
    image_tensor2D = torch.Tensor(image2D)
    image_tensor2D = image_tensor2D.view((1, 3, nx, nx))
    image_tensor3D = torch.Tensor(image3D)
    image_tensor3D = image_tensor3D.view((1, 1, 3, nx, nx)).to(device)  
    output = unet_model(image_tensor2D, image_tensor3D).detach().cpu()
    output=F.softmax(output, dim=1)
    output = torch.max(output, 1)[1]
    output = output.numpy()
    
    new_image[:,:,indexSlice]=output
    indexSlice=indexSlice+1

    if indexSlice==thickness[indexImg]: 
        new_image = nib.Nifti1Image(new_image, nib.load(mskC[0]).affine)
        nib.save(new_image, image_save_path+mskC[indexImg][12:])
        indexSlice=0
        indexImg=indexImg+1
        new_image=np.zeros([nx,nx,thickness[indexImg]])


IndexError: list index out of range

## Collect instant features and save into featureRecords.csv

In [10]:
def maskGeneration(image):
    mask=np.zeros(image.shape)
    for z in range(image.shape[2]): 
        check = False
        for j in range(50,image.shape[1]):
            for i in range(50,image.shape[0]):
                if image[i,j,z] == 1:
                    mask[i-20:i+20,j-20:j+20,z]=1
                    check=True
                    break
            if check==True:
                break
    return mask

In [11]:
import math
def instant_features(image):
    xResolution=nib.load(image_name).header['pixdim'][1]
    yResolution=nib.load(image_name).header['pixdim'][2]
    zResolution=nib.load(image_name).header['pixdim'][3]
    xyResolution=xResolution*yResolution
    
    zAxis=image.shape[2]
    image1=image.copy()
    image1[(image1!=1)]=0
    image1[(image1==1)]=1
    image1Agg=image1.sum(axis=2)*zResolution


    image2=image.copy()
    mask=maskGeneration(image2)
    image2[(image2!=2)]=0
    image2[(image2==2)]=1
    image2Agg=image2.sum(axis=2)
    IMG2AGG=(mask*image2).sum(axis=2)*zResolution

    image3=image.copy()
    image3[(image!=3)]=0
    image3[(image3==3)]=1
    image3Agg=image3.sum(axis=2)*zResolution
    
    p1, p2, p3, A1, A2, A3, circularity1, circularity2, circularity3 = [[] for i in range(9)]

    for i in range(zAxis):
        if np.sum(image1[:,:,i]) == 0:
            continue
        else:
            regions1 = regionprops(image1[:,:,i].astype(int))
            p1.append(regions1[0].perimeter*xResolution)
            A1.append(regions1[0].area*xyResolution) 
            circularity1.append(4*math.pi*A1[-1]/p1[-1]/p1[-1]) 
        if np.sum(image2[:,:,i]) == 0:
            continue
        else:    
            regions2 = regionprops(image2[:,:,i].astype(int)) 
            p2.append(regions2[0].perimeter*xResolution)
            A2.append(regions2[0].area*xyResolution)
            circularity2.append(4*math.pi*A2[-1]/p2[-1]/p2[-1]) 

        if np.sum(image3[:,:,i]) == 0:  
            continue
        else:
            regions3 = regionprops(image3[:,:,i].astype(int)) 
            p3.append(regions3[0].perimeter*xResolution)
            A3.append(regions3[0].area*xyResolution)
            circularity3.append(4*math.pi*A3[-1]/p3[-1]/p3[-1]) 
            

    maxThicknessRVC = np.max(image1Agg[(image1Agg>0)])
    minThicknessRVC = np.min(image1Agg[(image1Agg>0)])
    stdThicknessRVC = np.std(image1Agg[(image1Agg>0)])
    meanThicknessRVC = np.mean(image1Agg[(image1Agg>0)]) 
    
    maxThicknessLVC = np.max(image3Agg[(image3Agg>0)])
    minThicknessLVC = np.min(image3Agg[(image3Agg>0)])
    stdThicknessLVC = np.std(image3Agg[(image3Agg>0)])
    meanThicknessLVC = np.mean(image3Agg[(image3Agg>0)])
    
    maxThicknessLVM = np.max(image2Agg[(image2Agg>0)])
    minThicknessLVM = np.min(image2Agg[(image2Agg>0)])
    stdThicknessLVM = np.std(image2Agg[(image2Agg>0)])
    meanThicknessLVM = np.mean(image2Agg[(image2Agg>0)])
    
    meanCircularity1 = np.mean(circularity1)
    meanCircularity2 = np.mean(circularity2)
    meanCircularity3 = np.mean(circularity3)
    
    maxP1 = np.max(p1)
    meanP1 = np.mean(p1)
    maxP2 = np.max(p2)
    meanP2 = np.mean(p2)
    maxP3 = np.max(p3)
    meanP3 = np.mean(p3)
   
    meanMiddleThicknessLVM = np.mean(IMG2AGG[(IMG2AGG>0)]) 
    stdMiddleThicknessLVM = np.std(IMG2AGG[(IMG2AGG>0)])  
    
    if np.sum(image2[:,:,0])==0:
        RVCApicalLVM=np.sum(image1[:,:,1])*xyResolution
        RVCToLVC=np.sum(image1[:,:,1])/np.sum(image3[:,:,1])
    else:
        RVCApicalLVM=np.sum(image1[:,:,0])*xyResolution
        RVCToLVC=np.sum(image1[:,:,0])/np.sum(image3[:,:,0])
        
    return maxThicknessRVC, minThicknessRVC, stdThicknessRVC, meanThicknessRVC, maxThicknessLVC,\
    minThicknessLVC,stdThicknessLVC, meanThicknessLVC, maxThicknessLVM, minThicknessLVM,\
    stdThicknessLVM, meanThicknessLVM, meanMiddleThicknessLVM, stdMiddleThicknessLVM,meanCircularity1,\
    meanCircularity2,meanCircularity3,maxP1, meanP1, maxP2, meanP2, maxP3, meanP3, RVCApicalLVM,RVCToLVC
    


In [12]:
import pandas as pd
from skimage.measure import label, regionprops
from openpyxl import load_workbook
import io
##############################################################################
# Collect Data
mskC=glob.glob(os.path.join('datacollect/Saved/', 'patient???_frame??_gt.nii.gz'))
mskC.sort()
EDmaxThicknessRVC, EDminThicknessRVC, EDstdThicknessRVC, EDmeanThicknessRVC, EDmaxThicknessLVC = [[] for i in range(5)]
EDminThicknessLVC,EDstdThicknessLVC, EDmeanThicknessLVC, EDmaxThicknessLVM, EDminThicknessLVM = [[] for i in range(5)]
EDstdThicknessLVM, EDmeanThicknessLVM, EDmeanMiddleThicknessLVM, EDstdMiddleThicknessLVM, EDmeanCircularity1 = [[] for i in range(5)]
EDmeanCircularity2, EDmeanCircularity3, EDmaxP1, EDmeanP1, EDmaxP2 = [[] for i in range(5)]
EDmeanP2, EDmaxP3, EDmeanP3, EDRVCApicalLVM, EDRVCToLVC = [[] for i in range(5)]

ESmaxThicknessRVC, ESminThicknessRVC, ESstdThicknessRVC, ESmeanThicknessRVC, ESmaxThicknessLVC = [[] for i in range(5)]
ESminThicknessLVC, ESstdThicknessLVC, ESmeanThicknessLVC, ESmaxThicknessLVM, ESminThicknessLVM = [[] for i in range(5)]
ESstdThicknessLVM, ESmeanThicknessLVM, ESmeanMiddleThicknessLVM, ESstdMiddleThicknessLVM, ESmeanCircularity1 = [[] for i in range(5)]
ESmeanCircularity2, ESmeanCircularity3, ESmaxP1, ESmeanP1, ESmaxP2 = [[] for i in range(5)]
ESmeanP2, ESmaxP3, ESmeanP3, ESRVCApicalLVM, ESRVCToLVC = [[] for i in range(5)]

for i in range(200):
    image_name=mskC[i]
    image = nib.load(image_name).get_fdata()
    maxThicknessRVC, minThicknessRVC, stdThicknessRVC, meanThicknessRVC, maxThicknessLVC,\
    minThicknessLVC,stdThicknessLVC, meanThicknessLVC, maxThicknessLVM, minThicknessLVM,\
    stdThicknessLVM, meanThicknessLVM, meanMiddleThicknessLVM, stdMiddleThicknessLVM,meanCircularity1,\
    meanCircularity2,meanCircularity3,maxP1, meanP1, maxP2, meanP2, maxP3, meanP3, RVCApicalLVM,RVCToLVC\
    =instant_features(image)
    
    if i%2==0:
        EDmaxThicknessRVC.append(maxThicknessRVC)
        EDminThicknessRVC.append(minThicknessRVC)
        EDstdThicknessRVC.append(stdThicknessRVC)
        EDmeanThicknessRVC.append(meanThicknessRVC) 
        EDmaxThicknessLVC.append(maxThicknessLVC)
        EDminThicknessLVC.append(minThicknessLVC)
        EDstdThicknessLVC.append(stdThicknessLVC)
        EDmeanThicknessLVC.append(meanThicknessLVC)
        EDmaxThicknessLVM.append(maxThicknessLVM)
        EDminThicknessLVM.append(minThicknessLVM)
        EDstdThicknessLVM.append(stdThicknessLVM)
        EDmeanThicknessLVM.append(meanThicknessLVM)
        EDmeanMiddleThicknessLVM.append(meanMiddleThicknessLVM)
        EDstdMiddleThicknessLVM.append(stdMiddleThicknessLVM)
        EDmeanCircularity1.append(meanCircularity1)
        EDmeanCircularity2.append(meanCircularity2)
        EDmeanCircularity3.append(meanCircularity3)
        EDmaxP1.append(maxP1)
        EDmeanP1.append(meanP1)
        EDmaxP2.append(maxP2)
        EDmeanP2.append(meanP2)
        EDmaxP3.append(maxP3)
        EDmeanP3.append(meanP3)
        EDRVCApicalLVM.append(RVCApicalLVM)
        EDRVCToLVC.append(RVCToLVC)
        
        #print(maxP1)
        
    else:
        ESmaxThicknessRVC.append(maxThicknessRVC)
        ESminThicknessRVC.append(minThicknessRVC)
        ESstdThicknessRVC.append(stdThicknessRVC)
        ESmeanThicknessRVC.append(meanThicknessRVC) 
        ESmaxThicknessLVC.append(maxThicknessLVC)
        ESminThicknessLVC.append(minThicknessLVC)
        ESstdThicknessLVC.append(stdThicknessLVC)
        ESmeanThicknessLVC.append(meanThicknessLVC)
        ESmaxThicknessLVM.append(maxThicknessLVM)
        ESminThicknessLVM.append(minThicknessLVM)
        ESstdThicknessLVM.append(stdThicknessLVM)
        ESmeanThicknessLVM.append(meanThicknessLVM)
        ESmeanMiddleThicknessLVM.append(meanMiddleThicknessLVM)
        ESstdMiddleThicknessLVM.append(stdMiddleThicknessLVM)
        ESmeanCircularity1.append(meanCircularity1)
        ESmeanCircularity2.append(meanCircularity2)
        ESmeanCircularity3.append(meanCircularity3)
        ESmaxP1.append(maxP1)
        ESmeanP1.append(meanP1)
        ESmaxP2.append(maxP2)
        ESmeanP2.append(meanP2)
        ESmaxP3.append(maxP3)
        ESmeanP3.append(meanP3)
        ESRVCApicalLVM.append(RVCApicalLVM)
        ESRVCToLVC.append(RVCToLVC)

Book1.xlsx is the excel include subjects' group, height, NbFrame, Weights

In [13]:
##############################################################################
# Get thickness
imgC=glob.glob(os.path.join('datacollect', 'patient???_frame??.nii.gz'))
imgC.sort()
thickness=[]
for i in range(200):
        image_name=imgC[i]
        imageTemperate=nib.load(image_name)
        if (i%2==0):
            thickness.append(nib.load(image_name).header['pixdim'][3])
record = pd.read_excel('Book1.xlsx', index_col=0) 

##############################################################################
# Create excel form
 
data = pd.read_excel (r'Book1.xlsx')
df = pd.DataFrame(data, columns= None)
#print (df['ES'].values)
weight=df['Weight'].values[0:100]
height=df['Height'].values[0:100]
group=df['Group'].values[0:100]
BMI=weight/height/height
BSA=np.sqrt(weight*height/3600)
df = pd.DataFrame({'Height': height, 'Weight': weight, 'BMI': BMI, 'BSA': BSA,\
                   'Thickness': thickness,'EDmaxThicknessRVC': EDmaxThicknessRVC,\
                   'EDminThicknessRVC': EDminThicknessRVC, 'EDstdThicknessRVC': EDstdThicknessRVC,\
                   'EDmeanThicknessRVC': EDmeanThicknessRVC, 'EDmaxThicknessLVC': EDmaxThicknessLVC,\
                   'EDminThicknessLVC': EDminThicknessLVC,'EDstdThicknessLVC': EDstdThicknessLVC,\
                   'EDmeanThicknessLVC': EDmeanThicknessLVC, 'EDmaxThicknessLVM': EDmaxThicknessLVM,\
                   'EDminThicknessLVM': EDminThicknessLVM, 'EDstdThicknessLVM': EDstdThicknessLVM,\
                   'EDmeanThicknessLVM': EDmeanThicknessLVM, 'EDmeanMiddleThicknessLVM': EDmeanMiddleThicknessLVM,\
                   'EDstdMiddleThicknessLVM': EDstdMiddleThicknessLVM, 'EDmeanCircularity1': EDmeanCircularity1,\
                   'EDmeanCircularity2': EDmeanCircularity2, 'EDmeanCircularity3': EDmeanCircularity3,\
                   'EDmaxP1': EDmaxP1, 'EDmeanP1': EDmeanP1, 'EDmaxP2': EDmaxP2, 'EDmeanP2': EDmeanP2, \
                   'EDmaxP3': EDmaxP3, 'EDmeanP3': EDmeanP3, 'EDRVCApicalLVM': EDRVCApicalLVM,\
                   'EDRVCToLVC': EDRVCToLVC, 'ESmaxThicknessRVC': ESmaxThicknessRVC,\
                   'ESminThicknessRVC': ESminThicknessRVC, 'ESstdThicknessRVC': ESstdThicknessRVC,\
                   'ESmeanThicknessRVC': ESmeanThicknessRVC, 'ESmaxThicknessLVC': ESmaxThicknessLVC,\
                   'ESminThicknessLVC': ESminThicknessLVC,'ESstdThicknessLVC': ESstdThicknessLVC,\
                   'ESmeanThicknessLVC': ESmeanThicknessLVC, 'ESmaxThicknessLVM': ESmaxThicknessLVM,\
                   'ESminThicknessLVM': ESminThicknessLVM, 'ESstdThicknessLVM': ESstdThicknessLVM,\
                   'ESmeanThicknessLVM': ESmeanThicknessLVM, 'ESmeanMiddleThicknessLVM': ESmeanMiddleThicknessLVM,\
                   'ESstdMiddleThicknessLVM': ESstdMiddleThicknessLVM, 'ESmeanCircularity1': ESmeanCircularity1,\
                   'ESmeanCircularity2': ESmeanCircularity2, 'ESmeanCircularity3': ESmeanCircularity3,\
                   'ESmaxP1': ESmaxP1, 'ESmeanP1': ESmeanP1, 'ESmaxP2': ESmaxP2, 'ESmeanP2': ESmeanP2, \
                   'ESmaxP3': ESmaxP3, 'ESmeanP3': ESmeanP3, 'ESRVCApicalLVM': ESRVCApicalLVM,\
                   'Group': group})
## ExcelWriter for some reason uses writer.sheets to access the sheet.
## If you leave it empty it will not know that sheet Main is already there
## and will create a new sheet.


df.to_csv(r'featureRecords.csv', index = False)
df.head()




,Height,Weight,BMI,BSA,Thickness,EDmaxThicknessRVC,EDminThicknessRVC,EDstdThicknessRVC,EDmeanThicknessRVC,EDmaxThicknessLVC,...,ESmeanCircularity2,ESmeanCircularity3,ESmaxP1,ESmeanP1,ESmaxP2,ESmeanP2,ESmaxP3,ESmeanP3,ESRVCApicalLVM,Group
0,184.0,95.0,0.002806,2.203533,10.0,7.0,1.0,1.822130,3.470666,10.0,...,0.131196,0.923855,176.953319,123.534025,377.019336,354.167220,159.781746,148.205772,0.0,DCM
1,160.0,70.0,0.002734,1.763834,10.0,7.0,1.0,1.532438,2.969178,10.0,...,0.164151,0.922486,171.781746,82.884612,355.362482,298.101900,149.539105,119.408137,1004.0,DCM
2,165.0,77.0,0.002828,1.878608,10.0,8.0,1.0,1.835290,3.802650,9.0,...,0.141550,0.899708,202.166522,148.821789,375.362482,327.579962,161.195959,138.925004,7.0,DCM
3,159.0,46.0,0.001820,1.425365,10.0,7.0,1.0,1.719327,3.348715,9.0,...,0.125862,0.931113,171.195959,126.839989,354.534055,329.470346,149.781746,138.516134,777.0,DCM
4,165.0,77.0,0.002828,1.878608,10.0,8.0,1.0,1.935037,3.581096,10.0,...,0.178754,0.913312,188.994949,117.089919,366.676190,303.877084,148.367532,122.811183,86.0,DCM
